# Convert Tensorflow OD API model to Tensorflow Lite

## Installation

In [ ]:
!pip install tf-nightly -q

     |████████████████████████████████| 454.7MB 36kB/s 
     |████████████████████████████████| 471kB 35.3MB/s 
     |████████████████████████████████| 4.2MB 30.5MB/s 
     |████████████████████████████████| 5.5MB 20.5MB/s 
     |████████████████████████████████| 1.3MB 43.3MB/s 
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.38.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement keras-nightly~=2.5.0.dev, but you'll have keras-nightly 2.6.0.dev2021061200 which is incompatible.


In [ ]:
!pip install -q tflite_support

     |████████████████████████████████| 1.1MB 7.3MB/s 
     |████████████████████████████████| 194kB 26.1MB/s 


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2689, done.
remote: Counting objects: 100% (2689/2689), done.
remote: Compressing objects: 100% (2233/2233), done.
remote: Total 2689 (delta 683), reused 1247 (delta 423), pack-reused 0
Receiving objects: 100% (2689/2689), 32.67 MiB | 27.71 MiB/s, done.
Resolving deltas: 100% (683/683), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
pip install -q .

ERROR: tf-nightly 2.6.0.dev20210612 has requirement keras-nightly~=2.6.0.dev, but you'll have keras-nightly 2.5.0.dev2021032900 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.38.0 which is incompatible.
ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.30.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [ ]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2021-06-12 18:44:11.460702: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/count
2021-06-12 18:44:11.460781: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/count
2021-06-12 18:44:11.460804: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/api
2021-06-12 18:44:11.460835: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/api
2021-06-12 18:44:14.061006: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-12 18:44:14.061072: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host 

## Download model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
!rm ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

--2021-06-12 18:44:50--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  95.3MB/s    in 0.2s    

2021-06-12 18:44:50 (95.3 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]



In [ ]:
!ls -lah ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model

total 9.3M
drwxr-x--- 3 345018 89939 4.0K Jul 10  2020 .
drwxr-x--- 4 345018 89939 4.0K Jul 11  2020 ..
-rw-r----- 1 345018 89939 9.2M Jul 10  2020 saved_model.pb
drwxr-x--- 2 345018 89939 4.0K Jul 10  2020 variables


## Generate TensorFlow Lite model

### 1. Export a TFLite compatible model graph 

In [ ]:
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint \
    --output_directory `pwd` \
    --pipeline_config_path ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config

2021-06-12 18:46:50.073838: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/count
2021-06-12 18:46:50.073921: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/count
2021-06-12 18:46:50.073936: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/api
2021-06-12 18:46:50.073948: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/api
2021-06-12 18:46:52.191456: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-12 18:46:52.191511: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host 

In [ ]:
!ls -lah /content/saved_model/

total 9.8M
drwxr-xr-x 4 root root 4.0K Jun 12 18:47 .
drwxr-xr-x 1 root root 4.0K Jun 12 18:47 ..
drwxr-xr-x 2 root root 4.0K Jun 12 18:47 assets
-rw-r--r-- 1 root root 9.8M Jun 12 18:47 saved_model.pb
drwxr-xr-x 2 root root 4.0K Jun 12 18:47 variables


### 2. Convert model to Tensorflow Lite

In [ ]:
import tensorflow as tf

# Convert the model to TF lite
converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Serialize the model
open('model.tflite', 'wb').write(tflite_model)

4298944

In [ ]:
!ls -lah model.tflite

-rw-r--r-- 1 root root 4.1M Jun 12 18:48 model.tflite


### 3. Add Metadata

from https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/convert_odt_model_to_TFLite.ipynb

In [ ]:
from object_detection.utils import label_map_util

# Download the COCO dataset label map that was used to trained the SSD MobileNet V2 FPNLite 640x640 model
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt -q

# We need to convert the Object Detection API's labelmap into what the Task API needs:
# a txt file with one class name on each line from index 0 to N.
# The first '0' class indicates the background.
# This code assumes COCO detection which has 90 classes, you can write a label
# map file for your model if re-trained.
_ODT_LABEL_MAP_PATH = 'mscoco_label_map.pbtxt'
_TFLITE_LABEL_PATH = "/content/tflite_label_map.txt"

category_index = label_map_util.create_category_index_from_labelmap(
    _ODT_LABEL_MAP_PATH)
f = open(_TFLITE_LABEL_PATH, 'w')
for class_id in range(1, 91):
  if class_id not in category_index:
    f.write('???\n')
    continue
  name = category_index[class_id]['name']
  f.write(name+'\n')
f.close()

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

_TFLITE_MODEL_WITH_METADATA_PATH = "/content/model_with_metadata.tflite"

writer = object_detector.MetadataWriter.create_for_inference(
    writer_utils.load_file('model.tflite'), input_norm_mean=[127.5], 
    input_norm_std=[127.5], label_file_paths=[_TFLITE_LABEL_PATH])
writer_utils.save_file(writer.populate(), _TFLITE_MODEL_WITH_METADATA_PATH)

In [ ]:
from tflite_support import metadata

displayer = metadata.MetadataDisplayer.with_model_file(_TFLITE_MODEL_WITH_METADATA_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("=============================")
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

Metadata populated:
{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
    